In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import imutils
import glob
import cv2
import shutil
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import time
import fastbook
fastbook.setup_book()
from fastbook import *
pd.set_option('display.max_columns', 500)
tqdm_notebook().pandas()

C:\Users\mayan\AppData\Local\Temp/ipykernel_17156/1084472030.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

In [3]:
face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [4]:
def face_extactor(origin,destination,fc):
    img=cv2.imread(origin,1)
    img=imutils.resize(img,width=200)
    H,W,_=img.shape
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    face_coord=fc.detectMultiScale(gray,1.2,10,minSize=(50,50))
    if len(face_coord)==1:
        X,Y,w,h=face_coord[0]
    elif len(face_coord)==0:
        return None
    else:
        max_val=0
        max_idx=0
        for idx in range(len(face_coord)):
            _, _, w_i, h_i=face_coord[idx]
            if w_i*h_i>max_val:
                max_idx=idx
                max_val=w_i*h_i
            else:
                pass
            X,Y,w,h=face_coord[max_idx]
    img_cp=img[
        max(0,Y-int(0.35*h)): min(Y+int(1.35*h),H),
        max(0,X-int(w*0.35)): min(X+int(1.35*w),W)
    ].copy()
    cv2.imwrite(destination,img_cp)

In [5]:
path="F:/Deep Learning/FastAI/Datasets/CelebFace/img_align_celeba/img_align_celeba"
item_list=glob.glob("F:/Deep Learning/FastAI/Datasets/CelebFace/img_align_celeba/img_align_celeba/*.jpg")
print(len(item_list))

202599


In [6]:
for org in tqdm_notebook(item_list):
    face_extactor(origin=org,destination=path+org.split('/')[-1],fc=face_cascade)

C:\Users\mayan\AppData\Local\Temp/ipykernel_17156/1487666146.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for org in tqdm_notebook(item_list):


  0%|          | 0/202599 [00:00<?, ?it/s]

In [7]:
## Findign all the images and separating in training and validation
item_list = glob.glob(path+'*.jpg')

for idx in tqdm_notebook(range(1,202600)):
    if idx <= 182637:
        destination = path+'training/'
    else:
        destination = path+'validation/'
    try:
        shutil.move(
            path+str(idx).zfill(6)+'.jpg', 
            destination+str(idx).zfill(6)+'.jpg'
        )
    except:
        pass

C:\Users\mayan\AppData\Local\Temp/ipykernel_17156/2073080951.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(1,202600)):


  0%|          | 0/202599 [00:00<?, ?it/s]

In [8]:
## Combining all label attributes
label_df = pd.read_csv("F:/Deep Learning/FastAI/Datasets/CelebFace/list_attr_celeba.csv")
column_list = pd.Series(list(label_df.columns)[1:])

def label_generator(row):
    return(' '.join(column_list[[True if i==1 else False for i in row[column_list]]]))

label_df['label'] = label_df.progress_apply(lambda x: label_generator(x), axis=1)
label_df = label_df.loc[:,['image_id','label']]
label_df.to_csv("F:/Deep Learning/FastAI/Datasets/CelebFace/labels.csv")

  0%|          | 0/202599 [00:00<?, ?it/s]

In [9]:
## Attachhing label to correct file names
item_list = glob.glob("F:/Deep Learning/FastAI/Datasets/CelebFace/img_align_celeba/img_align_celeba/*/*.jpg")
item_df = pd.DataFrame({'image_name':pd.Series(item_list).apply(lambda x: '/'.join(x.split('/')[-2]))})
item_df['image_id'] = item_df.image_name.apply(lambda x: x.split('/')[1])

C:\Users\mayan\AppData\Local\Temp/ipykernel_17156/3810645426.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  item_df = pd.DataFrame({'image_name':pd.Series(item_list).apply(lambda x: '/'.join(x.split('/')[-2]))})
